In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv("../.env", override=True)

True

In [2]:
os.environ.get("PINECONE_ENV")

'gcp-starter'

In [7]:
import pinecone

pinecone.init(
    api_key=os.environ.get("PINECONE_API_KEY"),
    environment=os.environ.get("PINECONE_ENV")
)
pinecone.info.version()

VersionResponse(server='2.0.11', client='2.2.2')

In [9]:
pinecone.list_indexes()

[]

In [10]:
index_name = 'langchain-pinecone'
if index_name not in pinecone.list_indexes():
    print(f'Creating index {index_name} ....')
    pinecone.create_index(index_name, dimension=1536, metric='cosine', pods=1, pod_type='p1.x2')

Creating index langchain-pinecone ....


In [11]:
pinecone.describe_index(index_name)

IndexDescription(name='langchain-pinecone', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [13]:
pinecone.Index(index_name).describe_index_stats()

### Pinecone allows us to partition the vectors in an index into namespaces
### Querying & other operations are then limited to one namespace

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

### Upsert

In [15]:
import random
index = pinecone.Index(index_name)


vectors = [[random.random() for _ in range(1536)] for v in range(5)]
ids = list('abcde')

index.upsert(vectors = zip(ids, vectors))

{'upserted_count': 5}

In [16]:
index.upsert(vectors = [('c', [0.3]*1536)])

{'upserted_count': 1}

In [19]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 5e-05,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}

### Fetch

In [17]:
index.fetch(ids=['c', 'd'])

{'namespace': '',
 'vectors': {'c': {'id': 'c',
                   'metadata': {},
                   'values': [0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
       

### Query

In [24]:
index.query(
    vector = [[random.random() for _ in range(1536)] for v in range(1) ], 
    top_k=3,
    include_values=False
)

{'matches': [{'id': 'c', 'score': 0.86101, 'values': []},
             {'id': 'b', 'score': 0.754522204, 'values': []},
             {'id': 'e', 'score': 0.748517573, 'values': []}],
 'namespace': ''}

In [ ]:
index.query()

### Delete

In [27]:
index.delete(ids=list('abcde'))

{}

In [28]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}